## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['KIST 김영수 박사 유전자 변이로 모습을 바꾼 암세포도 감쪽같이 타격하는 표적항암제를 만들 수 있는 기초 연구 성과를 내놓은 KIST 치매DTC융합연구단 김영수 박사',
 'KIST 연구진, 유전자 변이 영향없는 새 표적 단백질 발견',
 '유전자 변이로 모습을 바꾼 암세포도 감쪽같이 타격하는 표적항암제를 만들 수 있는 기초 연구 성과가 국내에서 나왔다',
 "암 치료에서 '마법의 탄환'으로 꼽히지만, 유전자 변이 현상 앞에서는 '오발탄'이 되던 종전 표적항암제의 한계를 극복할 수 있는 성과라 주목된다",
 "한국과학기술연구원(KIST)은 치매DTC융합연구단 김영수 박사팀이 뇌종양 세포의 표면에서 과발현하는 단백질인 'GRP78'이 암의 전이를 조절하는 기능이 있다는 사실을 규명했다고 13일 밝혔다",
 '이 GRP78은 세포막에 발현되는 단백질이라 유전자 변이의 영향을 받지 않는다',
 "이 때문에 이 GRP78을 '표적'으로 삼는 치료제를 만들면 유전자 변이로 암세포가 돌변해도 목표를 놓치지 않고 정밀 타격을 할 수 있다고 김 박사팀은 전했다",
 '연구진은 "GRP78을 표적항암제로 억제하면 뇌종양 전이를 막고 암 치료도 되는 두 가지 효과가 나타나는 것으로 밝혀졌다"며 "향후 추가 연구를 통해 뇌종양 외에 다른 암에도 이번 성과를 적용할 수 있을 것으로 기대된다"고 설명했다',
 '표적항암제는 특정 암세포만 골라 공격하는 약으로, 인체의 다른 정상 세포를 해치지 않아 탈모·구토 등의 부작용이 없다는 장점 덕분에 국내외 의약계에서 각광을 받고 있다',
 "만성골수성백혈병 치료제인 '글리백'이 대표적 예다",
 "이번 연구 성과는 온라인 학술지인 '사이언티픽 리포츠' 7일 자에 게재됐다",
 "연구는 'KIST 영 팰로우십' 프로그램의 지원 아래 이뤄졌다",
 '‘알록달록한 과녁 한가운데 이중나선 모양의 DNA가 꽂혀 있다',
 '기초과학연구원(IBS) 유전체교정 연구단이 제작한 이 일러스트는 크리스퍼 유전자 가위의 정확성을 비교한 그림이다',
 '유

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [ ]:
okt = Okt()

In [ ]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
    return okt.morphs(sentence)

In [ ]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [ ]:
corpus_tokenized

[['KIST',
  '김영수',
  '박사',
  '유전자',
  '변',
  '이',
  '로',
  '모습',
  '을',
  '바꾼',
  '암세포',
  '도',
  '감쪽',
  '같이',
  '타격',
  '하는',
  '표적',
  '항암제',
  '를',
  '만들',
  '수',
  '있는',
  '기초',
  '연구',
  '성과',
  '를',
  '내놓은',
  'KIST',
  '치매',
  'DTC',
  '융합',
  '연',
  '구단',
  '김영수',
  '박사'],
 ['KIST', '연', '구진', ',', '유전자', '변', '이', '영향', '없는', '새', '표적', '단백질', '발견'],
 ['유전자',
  '변',
  '이',
  '로',
  '모습',
  '을',
  '바꾼',
  '암세포',
  '도',
  '감쪽',
  '같이',
  '타격',
  '하는',
  '표적',
  '항암제',
  '를',
  '만들',
  '수',
  '있는',
  '기초',
  '연구',
  '성과',
  '가',
  '국내',
  '에서',
  '나왔다'],
 ['암',
  '치료',
  '에서',
  "'",
  '마법',
  '의',
  '탄환',
  "'",
  '으로',
  '꼽히지만',
  ',',
  '유전자',
  '변',
  '이',
  '현상',
  '앞',
  '에서는',
  "'",
  '오발탄',
  "'",
  '이',
  '되던',
  '종전',
  '표적',
  '항암제',
  '의',
  '한계',
  '를',
  '극복',
  '할',
  '수',
  '있는',
  '성과',
  '라',
  '주목',
  '된다'],
 ['한국',
  '과학기술',
  '연구원',
  '(',
  'KIST',
  ')',
  '은',
  '치매',
  'DTC',
  '융합',
  '연',
  '구단',
  '김영수',
  '박사',
  '팀',
  '이',
  '뇌종양',
  '세포',
  '의',


# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [ ]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count=0, window=3)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4968


In [ ]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs =5 )

(100677, 123005)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar("추석")

[('없이도', 0.33807769417762756),
 ('되게', 0.3295366168022156),
 ('내디딘', 0.28317195177078247),
 ('쫓아가는', 0.2732948362827301),
 ('250만', 0.2724350094795227),
 ('꾸렸다고', 0.27131298184394836),
 ('멀거나', 0.2680261731147766),
 ('달린', 0.2679099440574646),
 ('정보통', 0.2661566138267517),
 ('화대', 0.2642405927181244)]

In [ ]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(min_count=0, window=3)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4968


In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 32433


ValueError: ignored

In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(119483, 123005)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar("추석")

[('태음', 0.4432578682899475),
 ('수원부', 0.4051220715045929),
 ('반투명', 0.39532795548439026),
 ('밀라노', 0.38046523928642273),
 ('명조', 0.36522504687309265),
 ('포유', 0.34700870513916016),
 ('없이도', 0.3462407886981964),
 ('오강', 0.34230777621269226),
 ('공주군', 0.3398020565509796),
 ('슬퍼하', 0.3364577889442444)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4968

In [ ]:
unique_text[0]

'아직'

In [ ]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [ ]:
X, t = make_swiss_roll(n_samples = 1000, noise = 0)
color = (t - t.min()) / (t.max() - t.min())

In [ ]:
pca = PCA(random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [ ]:
# 시각화해보세요!

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [ ]:
# 시각화해보세요!

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 500).fit_transform(vector)

In [ ]:
# 시각화해보세요!

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!